In [ ]:
from matminer.datasets import load_dataset
import pandas as pd
from pymatgen.ext.matproj import MPRester
from ase import Atoms
from pymatgen.io.ase import AseAtomsAdaptor
from dscribe.descriptors import SOAP
import numpy as np
import pickle
import gzip
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from pymatgen.core import Composition
from mendeleev import element
from tqdm import tqdm  # optional, for progress bar


## Band Gap energy
Load in the dft and experimental data. Convert dft compositional data to formula to match the experimental formula.

In [30]:
# load experimental data
df_exp = load_dataset('expt_gap_kingsbury')
df_exp = df_exp.dropna(subset=['likely_mpid'])
print(df_exp.shape)
display(df_exp.head())

# load DFT data
df_dft = load_dataset('jarvis_dft_3d')
df_dft = df_dft.drop(columns=['epsilon_x opt', 'epsilon_y opt', 'epsilon_z opt', 'shear modulus', 'bulk modulus', 'jid', 'epsilon_x tbmbj', 'epsilon_y tbmbj', 'epsilon_z tbmbj'])
df_dft['formula'] = df_dft['composition'].apply(lambda x: x.reduced_formula) # convert composition to formula
df_dft = df_dft.dropna()
print(df_dft.shape)
display(df_dft.head())

(2481, 3)


,likely_mpid,expt_gap,formula
1,mp-29717,0.00,Ag(W3Br7)2
5,mp-23558,0.00,Ag2BiO3
6,mp-9900,1.98,Ag2GeS3
8,mp-23485,2.47,Ag2HgI4
9,mp-1203404,3.06,Ag2Mo(I2O7)2


/Users/cadenmyers/miniconda3/envs/ml4ms/lib/python3.13/site-packages/pymatgen/core/periodic_table.py:289: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
/Users/cadenmyers/miniconda3/envs/ml4ms/lib/python3.13/site-packages/pymatgen/core/periodic_table.py:289: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(
/Users/cadenmyers/miniconda3/envs/ml4ms/lib/python3.13/site-packages/pymatgen/core/periodic_table.py:289: UserWarning: No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  warnings.warn(


(7349, 8)


,structure,e_form,structure initial,gap tbmbj,mpid,gap opt,composition,formula
59,"[[ 8.9342299 3.79961599 -6.97400918] Mg, [ 3...",-2.896,"[[ 8.9825888 3.8061472 -7.01147537] Mg, [ 3...",7.8043,mp-6596,5.2711,"(Mg, Al, P, O)",MgAlPO5
68,"[[1.25762093 4.13616104 2.44785671] K, [3.7426...",-2.657,"[[1.26002845 4.20573168 2.5197491 ] K, [3.7544...",8.3622,mp-7324,5.6141,"(K, Be, P, O)",KBePO4
75,"[[3.1506963 4.30596127 3.56982179] As, [0.384...",0.025,"[[3.17888147 4.36479751 3.69558323] As, [0.381...",0.6495,mp-158,0.0221,(As),As
80,"[[0.37213617 3.67987527 0.87369073] H, [1.4889...",-1.184,"[[0.34436223 3.69323659 0.87470146] H, [1.4990...",0.0083,mp-24242,0.0038,"(H, O, F, Cu)",CuHOF
83,"[[ 0.92796672 1.82915421 10.16747299] O, [2.7...",-2.086,"[[ 0.938753 1.84002706 10.93783201] O, [2.8...",2.4642,mp-510584,1.9202,"(O, Mo)",MoO3


Create a dataframe of dft data for MP-ID's that are in both the experimental and dft datasets.

In [31]:
df_dft_filtered = df_dft[df_dft['mpid'].isin(df_exp['likely_mpid'])].reset_index(drop=True)
print(df_dft_filtered.shape)
mpids = df_dft['mpid'].tolist() # list of mpids
print(mpids[0:5])

(329, 8)
['mp-6596', 'mp-7324', 'mp-158', 'mp-24242', 'mp-510584']


## Generate the SOAP descriptor

The **SOAP (Smooth Overlap of Atomic Positions)** descriptor is a powerful tool used in computational materials science to represent the local atomic environment of a material. It encodes information about the local symmetry and structure of atoms in a material, making it suitable for use in machine learning models that predict material properties.

### Key Features:
- **Local Environment Representation**: The SOAP descriptor captures the atomic environment around each atom in a structure, using a smooth overlap of atomic positions.
- **Spherical Harmonics**: The descriptor uses spherical harmonics to represent angular information, allowing it to effectively capture local symmetries.
- **Radial Basis Functions**: The radial part of the descriptor is represented using radial basis functions, which help model the distances between atoms.
- **Species Specific**: The descriptor can incorporate the chemical species present in the structure, allowing it to adapt to different materials.

### How It Works:
1. **Structure Representation**: The local atomic environment of each atom is represented by a combination of radial and angular functions.
2. **Cutoff Radius**: Only atoms within a certain cutoff radius contribute to the descriptor, ensuring computational efficiency.
3. **Smearing**: Gaussian smearing is used to smooth the atomic positions, ensuring a continuous representation.
4. **Periodic Structures**: SOAP can be used to model periodic structures such as crystals, making it suitable for large-scale materials simulations.


In [ ]:

def generate_soap_descriptor(mp_id, API_KEY=API_KEYcm):
    # 1. Connect to Materials Project
    mpr = MPRester(API_KEY)
    
    # 2. Fetch structure by MP-ID
    structure_pmg = mpr.get_structure_by_material_id(mp_id)

    # 3. Convert pymatgen structure to ASE Atoms
    ase_structure = AseAtomsAdaptor.get_atoms(structure_pmg)

    # 4. Check if species from provided list matches with the structure
    structure_species = list(set(ase_structure.get_chemical_symbols()))  # Unique elements in the structure
    
    # 5. Set up SOAP descriptor
    soap = SOAP(
        species=structure_species,   # List of elements present (user-specified)
        periodic=True,          # Is the structure periodic? (crystals = True, molecules = False)
        r_cut=5.0,              # Cutoff radius (Å)
        n_max=8,                # Number of radial basis functions
        l_max=6,                # Maximum degree of spherical harmonics
        sigma=0.5,              # Width (smearing) of Gaussians placed on atoms (Å)
        sparse=False            # Should output be dense NumPy array? (False = full array; True = sparse matrix for memory saving)
    )

    # 6. Create SOAP descriptors
    soap_descriptors = soap.create(ase_structure)

    return soap_descriptors

# Example usage
# soap_descriptor = generate_soap_descriptor(mp_id=mpids[0])

# print(soap_descriptor.shape)



## Calculate Soap Descriptors
Below code generates soap descriptors for all mpids in the df_dft dataset and creates a dataframe

In [5]:
# Generate SOAP descriptors for all mpids
soap_dict = {}
for i, mpid in enumerate(mpids):
    try:
        soap_descriptor = generate_soap_descriptor(mp_id=mpid)
        soap_dict[df_dft['formula'].iloc[i]] = soap_descriptor
        soap_dict[df_dft['mpid'].iloc[i]] = soap_descriptor
    except Exception as e:
        print(f"Skipping {mpid} due to error: {e}")


df_soap = pd.DataFrame({
    'formula': soap_dict.keys(),
    'soap_descriptor': soap_dict.values()
})
df_soap = df_soap.merge(df_dft[['formula', 'mpid']], on='formula', how='left')
df_soap.drop_duplicates(subset=['formula'], inplace=True)
df_soap.dropna(inplace=True)
df_soap.reset_index(drop=True, inplace=True)
print(df_soap.shape)
save_path = '/Users/cadenmyers/billingelab/dev/ml4ms_bandgap_final/data/soap_descriptors_raw.pkl.gz'
# # Save as compressed pickle
# with gzip.open(save_path, 'wb') as f:
#     pickle.dump(df_soap, f)

/Users/cadenmyers/miniconda3/envs/ml4ms/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 16448.25it/s]
Retrieving MaterialsDoc documents: 0it [00:00, ?it/s]


Skipping mp-510584 due to error: 'list' object has no attribute 'is_ordered'


Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 14027.77it/s]
Retrieving MaterialsDoc documents: 0it [00:00, ?it/s]


Skipping mp-714976 due to error: 'list' object has no attribute 'is_ordered'


Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 5440.08it/s]
Retrieving MaterialsDoc documents: 0it [00:00, ?it/s]


...

Skipping mp-25643 due to error: 'list' object has no attribute 'is_ordered'


Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 16448.25it/s]
Retrieving MaterialsDoc documents: 0it [00:00, ?it/s]


Skipping mp-31621 due to error: 'list' object has no attribute 'is_ordered'


Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 2565.32it/s]
Retrieving MaterialsDoc documents: 0it [00:00, ?it/s]


Skipping mp-782689 due to error: 'list' object has no attribute 'is_ordered'
Skipping auid-01b8499b8bba44df due to error: auid-01b8499b8bba44df is not formatted correctly!
Skipping auid-0259d68674b800c3 due to error: auid-0259d68674b800c3 is not formatted correctly!
Skipping auid-0361058f418da352 due to error: auid-0361058f418da352 is not formatted correctly!
Skipping auid-118e55383622ad35 due to error: auid-118e55383622ad35 is not formatted correctly!
Skipping auid-12f587b708c648f0 due to error: auid-12f587b708c648f0 is not formatted correctly!
Skipping auid-134f09989090f022 due to error: auid-134f09989090f022 is not formatted correctly!
Skipping auid-161a646146669ff6 due to error: auid-161a646146669ff6 is not formatted correctly!
Skipping auid-18c521390914a2e9 due to error: auid-18c521390914a2e9 is not formatted correctly!
Skipping auid-29b96b91e46e74a5 due to error: auid-29b96b91e46e74a5 is not formatted correctly!
Skipping auid-2bf2603e574aa6bf due to error: auid-2bf2603e574aa6bf i

In [17]:
df_soap.head(10)
df_dft_dedup = df_dft.drop_duplicates(subset='formula', keep='first')
df_soap_raw = df_soap.merge(df_dft_dedup[['formula', 'gap opt']], on='formula', how='left')
print(df_soap_raw.shape)
df_soap_raw.head(10)


(5358, 4)


,formula,soap_descriptor,mpid,gap opt
0,MgAlPO5,"[[7.834700877986906e-05, -0.000248821014237357...",mp-6596,5.2711
1,KBePO4,"[[1.224395792223083e-07, -5.548160208843811e-0...",mp-7324,5.6141
2,As,"[[0.18087236729129627, 0.4693173276837286, 0.5...",mp-158,0.0221
3,CuHOF,"[[8.408499979827601e-07, -6.4356783799753e-06,...",mp-24242,0.0038
4,AgI,"[[0.18083827914220227, 0.4690888310062766, 0.5...",mp-567809,1.2599
5,ZrBrN,"[[1.6087614466449077e-05, -5.929285214217304e-...",mp-570157,1.8723
6,InBr3,"[[3.8849510688717664e-07, 1.0707179561579893e-...",mp-570219,2.3104
7,YI3,"[[0.18204641291148752, 0.4670436680038411, 0.5...",mp-571442,2.6760
8,AlHO2,"[[5.891756608461837e-06, -2.8729819768613437e-...",mp-625055,5.1686
9,BiOF,"[[3.730708030672268e-06, -2.0038238223605248e-...",mp-759883,2.9829


In [23]:

def pad_or_truncate_descriptor(descriptor, target_shape=(64, 800)):
    """Ensure a SOAP descriptor has the shape (64, 800) by truncating or zero-padding."""
    if descriptor is None:
        return np.zeros(target_shape)

    descriptor = np.array(descriptor)
    padded = np.zeros(target_shape)

    # Determine actual slice size
    rows = min(target_shape[0], descriptor.shape[0])
    cols = min(target_shape[1], descriptor.shape[1])

    padded[:rows, :cols] = descriptor[:rows, :cols]
    return padded

# Apply to the 'soap_descriptor' column and store in new column 'padded_soap'
# df_soap_raw['padded_soap'] = df_soap_raw['soap_descriptor'].apply(pad_or_truncate_descriptor)
# df_soap_raw.drop(columns=['soap_descriptor'], inplace=True)
print(df_soap_raw.shape)
display(df_soap_raw.head(10))

(5358, 4)


,formula,mpid,gap opt,padded_soap
0,MgAlPO5,mp-6596,5.2711,"[[7.834700877986906e-05, -0.000248821014237357..."
1,KBePO4,mp-7324,5.6141,"[[1.224395792223083e-07, -5.548160208843811e-0..."
2,As,mp-158,0.0221,"[[0.18087236729129627, 0.4693173276837286, 0.5..."
3,CuHOF,mp-24242,0.0038,"[[8.408499979827601e-07, -6.4356783799753e-06,..."
4,AgI,mp-567809,1.2599,"[[0.18083827914220227, 0.4690888310062766, 0.5..."
5,ZrBrN,mp-570157,1.8723,"[[1.6087614466449077e-05, -5.929285214217304e-..."
6,InBr3,mp-570219,2.3104,"[[3.8849510688717664e-07, 1.0707179561579893e-..."
7,YI3,mp-571442,2.6760,"[[0.18204641291148752, 0.4670436680038411, 0.5..."
8,AlHO2,mp-625055,5.1686,"[[5.891756608461837e-06, -2.8729819768613437e-..."
9,BiOF,mp-759883,2.9829,"[[3.730708030672268e-06, -2.0038238223605248e-..."


In [8]:
# # Save the padded descriptors
# save_path = '/Users/cadenmyers/billingelab/dev/ml4ms_bandgap_final/data/padded_soap_descriptors.pkl.gz'
# with gzip.open(save_path, 'wb') as f:
#     pickle.dump(df_soap_raw, f)

NameError: name 'soap_df' is not defined

In [9]:
def find_largest_shape_indices_and_values(soap_descriptors):
    """
    Find the indices and values of the largest x and y dimensions among SOAP descriptors.

    Parameters:
    soap_descriptors (list or pd.Series): List of numpy arrays representing SOAP descriptors.

    Returns:
    tuple: (index_max_x, max_x, index_max_y, max_y)
    """
    max_x = 0
    max_y = 0
    index_max_x = -1
    index_max_y = -1

    for i, desc in enumerate(soap_descriptors):
        if desc is not None and len(desc.shape) >= 2:
            x, y = desc.shape
            if x > max_x:
                max_x = x
                index_max_x = i
            if y > max_y:
                max_y = y
                index_max_y = i

    return index_max_x, max_x, index_max_y, max_y


index_max_x, max_x, index_max_y, _ = find_largest_shape_indices_and_values(df_soap['soap_descriptor'])



def zero_pad_soap_descriptors(soap_descriptors, target_shape):
    """
    Zero-pad each SOAP descriptor to the target shape.

    Parameters:
    soap_descriptors (list or pd.Series): List of numpy arrays representing SOAP descriptors.
    target_shape (tuple): (target_x, target_y) shape to pad to.

    Returns:
    list: List of zero-padded numpy arrays.
    """
    padded_descriptors = []

    for desc in soap_descriptors:
        if desc is None:
            padded = np.zeros(target_shape)
        else:
            padded = np.zeros(target_shape)
            x_size = min(desc.shape[0], target_shape[0])
            y_size = min(desc.shape[1], target_shape[1]) if len(desc.shape) > 1 else 1

            # Copy data into padded array
            if len(desc.shape) == 1:
                padded[:x_size, 0] = desc[:x_size]
            else:
                padded[:x_size, :y_size] = desc[:x_size, :y_size]

        padded_descriptors.append(padded)

    return padded_descriptors

# limit `max_y`to 800 because not much info pasted 800
# max_y = 800
# print(max_x, max_y)
# padded_descriptors = zero_pad_soap_descriptors(df_soap['soap_descriptor'], (max_x, max_y))

64 800


# Get periodic table properties
get the properties and save them in a df

In [ ]:
# Choose your formula
formula_list = soap_df['formula'].tolist()

# Properties to extract
properties = [
    "electronegativity",
    "atomic_radius",
    "ionenergies",
    "covalent_radius",
    "nvalence",  # number of valence electrons
]


# Aggregation function per formula
def get_composition_features(formula):
    try:
        comp = Composition(formula)
        el_amt_dict = comp.get_el_amt_dict()
        total_atoms = comp.num_atoms
        
        data = {prop: [] for prop in properties}
        fractions = []

        for el, amt in el_amt_dict.items():
            try:
                elem = element(el)
            except:
                continue

            frac = amt / total_atoms
            fractions.append(frac)

            for prop in properties:
                val = getattr(elem, prop)

                if callable(val):
                    val = val()

                # Special case: ionenergies is a dict
                if isinstance(val, dict):
                    val = val.get(1, np.nan)
                elif val is None:
                    val = np.nan

                data[prop].append(val)

        # Compute aggregated features
        features = {"formula": formula}
        for prop in properties:
            values = np.array(data[prop])
            fracs = np.array(fractions)
            weighted_avg = np.nansum(values * fracs)
            max_val = np.nanmax(values)
            min_val = np.nanmin(values)
            std_val = np.nanstd(values)

            features[f"{prop}_mean"] = weighted_avg
            features[f"{prop}_max"] = max_val
            features[f"{prop}_min"] = min_val
            # features[f"{prop}_range"] = max_val - min_val
            features[f"{prop}_std"] = std_val

        return features

    except Exception as e:
        print(f"Failed on {formula}: {e}")
        return {"formula": formula}

# Run for all formulas with progress bar
# feature_list = [get_composition_features(f) for f in tqdm(formula_list)]
# features_df = pd.DataFrame(feature_list)
# print(features_df.shape)
# print(features_df.columns)
# features_df.head()

In [7]:
features_df = pd.read_excel('/Users/cadenmyers/billingelab/dev/ml4ms_bandgap_final/data/elemental_features.xlsx')

features_df.head()

,formula,electronegativity_mean,electronegativity_max,electronegativity_min,electronegativity_std,atomic_radius_mean,atomic_radius_max,atomic_radius_min,atomic_radius_std,ionenergies_mean,...,ionenergies_min,ionenergies_std,covalent_radius_mean,covalent_radius_max,covalent_radius_min,covalent_radius_std,nvalence_mean,nvalence_max,nvalence_min,nvalence_std
0,MgAlPO5,2.78875,3.44,1.31,0.815824,84.375,150.0,60.0,33.236840,11.526121,...,5.985769,2.902622,86.375000,139,63,28.752174,5.000000,6,2,1.581139
1,KBePO4,2.62000,3.44,0.82,0.960065,95.000,220.0,60.0,59.621200,11.231753,...,4.340664,3.337988,94.428571,196,63,48.512885,4.571429,6,1,2.061553
2,As,2.18000,2.18,2.18,0.000000,115.000,115.0,115.0,0.000000,9.788550,...,9.788550,0.000000,121.000000,121,121,0.000000,5.000000,5,5,0.000000
3,CuHOF,2.88000,3.98,1.90,0.858254,67.500,135.0,25.0,41.003049,13.091422,...,7.726380,3.466954,67.750000,112,32,28.603977,6.250000,11,1,3.561952
4,AgI,2.29500,2.66,1.93,0.365000,150.000,160.0,140.0,10.000000,9.013735,...,7.576234,1.437501,130.500000,133,128,2.500000,9.000000,11,7,2.000000


In [19]:
soap_df['soap_flat'] = soap_df['padded_soap'].apply(lambda x: x.flatten())
print(soap_df['soap_flat'].iloc[0].shape)
soap_df.head()

(51200,)


,formula,mpid,gap opt,padded_soap,soap_flat
0,MgAlPO5,mp-6596,5.2711,"[[7.834700877986906e-05, -0.000248821014237357...","[7.834700877986906e-05, -0.0002488210142373574..."
1,KBePO4,mp-7324,5.6141,"[[1.224395792223083e-07, -5.548160208843811e-0...","[1.224395792223083e-07, -5.548160208843811e-07..."
2,As,mp-158,0.0221,"[[0.18087236729129627, 0.4693173276837286, 0.5...","[0.18087236729129627, 0.4693173276837286, 0.55..."
3,CuHOF,mp-24242,0.0038,"[[8.408499979827601e-07, -6.4356783799753e-06,...","[8.408499979827601e-07, -6.4356783799753e-06, ..."
4,AgI,mp-567809,1.2599,"[[0.18083827914220227, 0.4690888310062766, 0.5...","[0.18083827914220227, 0.4690888310062766, 0.54..."


In [21]:
merged_df = pd.merge(soap_df, features_df, on='formula', how='left')
print(merged_df.shape)
merged_df.head()

(5358, 25)


,formula,mpid,gap opt,padded_soap,soap_flat,electronegativity_mean,electronegativity_max,electronegativity_min,electronegativity_std,atomic_radius_mean,...,ionenergies_min,ionenergies_std,covalent_radius_mean,covalent_radius_max,covalent_radius_min,covalent_radius_std,nvalence_mean,nvalence_max,nvalence_min,nvalence_std
0,MgAlPO5,mp-6596,5.2711,"[[7.834700877986906e-05, -0.000248821014237357...","[7.834700877986906e-05, -0.0002488210142373574...",2.78875,3.44,1.31,0.815824,84.375,...,5.985769,2.902622,86.375000,139,63,28.752174,5.000000,6,2,1.581139
1,KBePO4,mp-7324,5.6141,"[[1.224395792223083e-07, -5.548160208843811e-0...","[1.224395792223083e-07, -5.548160208843811e-07...",2.62000,3.44,0.82,0.960065,95.000,...,4.340664,3.337988,94.428571,196,63,48.512885,4.571429,6,1,2.061553
2,As,mp-158,0.0221,"[[0.18087236729129627, 0.4693173276837286, 0.5...","[0.18087236729129627, 0.4693173276837286, 0.55...",2.18000,2.18,2.18,0.000000,115.000,...,9.788550,0.000000,121.000000,121,121,0.000000,5.000000,5,5,0.000000
3,CuHOF,mp-24242,0.0038,"[[8.408499979827601e-07, -6.4356783799753e-06,...","[8.408499979827601e-07, -6.4356783799753e-06, ...",2.88000,3.98,1.90,0.858254,67.500,...,7.726380,3.466954,67.750000,112,32,28.603977,6.250000,11,1,3.561952
4,AgI,mp-567809,1.2599,"[[0.18083827914220227, 0.4690888310062766, 0.5...","[0.18083827914220227, 0.4690888310062766, 0.54...",2.29500,2.66,1.93,0.365000,150.000,...,7.576234,1.437501,130.500000,133,128,2.500000,9.000000,11,7,2.000000


In [ ]:
# # Save the merged DataFrame as a compressed pickle
# save_path = '/Users/cadenmyers/billingelab/dev/ml4ms_bandgap_final/data/soap_and_atomic_features.pkl.gz'
# with gzip.open(save_path, 'wb') as f:
#     pickle.dump(merged_df, f)